### Получение данных

Подключаемые модули

In [0]:
import os
from io import BytesIO

In [0]:
import pandas as pd
import pickle
from zipfile import ZipFile
from urllib.request import urlopen, urlretrieve

Подключение Gdrive

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Загрузка Cisco Umbrella Popularity List (legit)

In [0]:
if not os.path.isfile('/content/gdrive/My Drive/Colab Notebooks/Res/input data/top-1m.csv'):
    resp = urlopen('http://s3-us-west-1.amazonaws.com/umbrella-static/top-1m.csv.zip')
    zipfile = ZipFile(BytesIO(resp.read()))
    zipfile.extractall("/content/gdrive/My Drive/Colab Notebooks/Res/input data")

Загрузка Malware Domain by John Bambenek of Bambenek Consulting (dga)

In [5]:
urlretrieve('http://osint.bambenekconsulting.com/feeds/dga-feed.txt', '/content/gdrive/My Drive/Colab Notebooks/Res/input data/dga.csv')

('/content/gdrive/My Drive/Colab Notebooks/Res/input data/dga.csv',
 <http.client.HTTPMessage at 0x7f3750706a90>)

Конструкция данных обучения

In [0]:
training_data = {'legit': [], 'dga': []}

Выделение второго уровная доменного имени

In [0]:
domain_list = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Res/input data/top-1m.csv', names=['domain'])
domain_list['domain'] = domain_list['domain'].map(lambda x: x.split('.')[-2:]).map(lambda x: x[0])
domain_list['type'] = 0
training_data['legit'] = domain_list

In [8]:
training_data['legit']

,domain,type
1,netflix,0
2,netflix,0
3,netflix,0
4,netflix,0
5,google,0
...,...,...
999996,emltrk,0
999997,3000toys,0
999998,jakegame,0
999999,siteimproveanalytics,0


In [0]:
domain_list = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Res/input data/dga.csv', names=['domain', 'family', 'data'], skiprows = 14, index_col=False)
domain_list['domain'] = domain_list['domain'].map(lambda x: x.split('.')[0])
domain_list['family'] = domain_list['family'].map(lambda x: x.split(' ')[3])
domain_list['type'] = pd.factorize(domain_list.family)[0] + 1
training_data['dga'] = domain_list

In [11]:
training_data['dga']

,domain,family,data,type
0,yfxihttvspowlj,Cryptolocker,2019-11-14,1
1,ibontoytrpwule,Cryptolocker,2019-11-14,1
2,vnnvwjuhiroolh,Cryptolocker,2019-11-14,1
3,jceltexjevuxjr,Cryptolocker,2019-11-14,1
4,wodtwytwuxmrsi,Cryptolocker,2019-11-14,1
...,...,...,...,...
817626,tkdnd,unknowndropper,2019-11-16,53
817627,tkndd,unknowndropper,2019-11-16,53
817628,tnddk,unknowndropper,2019-11-16,53
817629,tndkd,unknowndropper,2019-11-16,53


Сохранение данных на диск

In [0]:
with open('/content/gdrive/My Drive/Colab Notebooks/Res/input data/training_data.pkl', 'wb') as f:
    pickle.dump(training_data, f, pickle.HIGHEST_PROTOCOL)